In [17]:
# https://towardsdatascience.com/pandas-groupby-a-simple-but-detailed-tutorial-314b8f37005d
# https://towardsdatascience.com/accessing-data-in-a-multiindex-dataframe-in-pandas-569e8767201d
# https://towardsdatascience.com/summarizing-data-with-pandas-crosstab-efc8b9abecf
# https://towardsdatascience.com/how-to-flatten-multiindex-columns-and-rows-in-pandas-f5406c50e569
# https://datascientyst.com/list-aggregation-functions-aggfunc-groupby-pandas/
# https://stackoverflow.com/questions/25929319/how-to-iterate-over-pandas-multiindex-dataframe-using-index
# https://stackoverflow.com/questions/24495695/pandas-get-unique-multiindex-level-values-by-label
# https://stackoverflow.com/questions/55706391/pandas-crosstab-on-multiple-columns-then-groupby

# https://matplotlib.org/stable/gallery/pyplots/pyplot_text.html#sphx-glr-gallery-pyplots-pyplot-text-py

In [18]:
import pandas as pd
import numpy as np
from myUtils import pickle_load, pickle_dump, symb_perf_stats_vectorized

path_dir = "C:/Users/ping/MyDrive/stocks/yfinance/"
path_data_dump = path_dir + "VSCode_dump/"

filename_symbols = path_data_dump + 'vg_symbols_4chars_max.csv'  # symbols text file
filename_pickled_df_OHLCVA_downloaded = 'df_OHLCVA_downloaded '  # OHLCVA downloaded from Yahoo
filename_pickled_df_adjOHLCV = 'df_adjOHLCV'  # adjusted OHLCV
filename_pickled_df_symbols_close = "df_symbols_close"  # symbols' adjusted close
filename_pickled_symbols_df_adjOHLCV =  'symbols_df_adjOHLCV'  # symbols in df_adjOHLCV

verbose = False  # True prints more output
look_back_days = -250  # subset df iloc days

In [19]:
print(f"Full path to pickled df_symbols_close:  {path_data_dump}{filename_pickled_df_symbols_close}")
df_close = pickle_load(path_data_dump, filename_pickled_df_symbols_close, verbose=verbose)
print(f"Full path to pickled df_OHLCVA_downloaded:  {path_data_dump}{filename_pickled_df_OHLCVA_downloaded}")
df_OHLCVA = pickle_load(path_data_dump, filename_pickled_df_OHLCVA_downloaded, verbose=verbose)

Full path to pickled df_symbols_close:  C:/Users/ping/MyDrive/stocks/yfinance/VSCode_dump/df_symbols_close
Full path to pickled df_OHLCVA_downloaded:  C:/Users/ping/MyDrive/stocks/yfinance/VSCode_dump/df_OHLCVA_downloaded 


In [20]:
# https://stackoverflow.com/questions/63826291/pandas-series-find-column-by-value
df = df_OHLCVA[look_back_days::]
df_v = df.xs('Volume', level=1, axis=1)  # select only Volume columns
rows, cols = np.where(df_v == 0)  # row index, column index where trading volumes are zero
idx_no_volume = list(set(cols))
idx_no_volume.sort()
symbols_no_volume = df_v.columns[idx_no_volume]
symbols_no_volume

Index(['AAIN', 'AAMC', 'ACAB', 'ACAD', 'ACAQ', 'ACBA', 'ACDI', 'ADAL', 'ADER',
       'ADOC',
       ...
       'WLYB', 'WMPN', 'WNNR', 'WQGA', 'WRAC', 'WTMA', 'WWAC', 'XFIN', 'XPER',
       'YNDX'],
      dtype='object', length=523)

In [21]:
df_c = df.xs('Close', level=1, axis=1)  # select only Close columns
df_c = df_c.fillna(0).copy()  # convert NaNs to zeros
rows, cols = np.where(df_c == 0)  # row index, column index where trading volumes are zero
idx_no_close = list(set(cols))
idx_no_close.sort()
symbols_no_close = df_c.columns[idx_no_close]
symbols_no_close

Index(['AAMC', 'ABM', 'ACAB', 'ACAQ', 'ACDI', 'ACLX', 'ADAL', 'ADRT', 'AEAE',
       'AFAC',
       ...
       'XRAY', 'XYL', 'YNDX', 'YUM', 'ZBH', 'ZBRA', 'ZEUS', 'ZGN', 'ZION',
       'ZNH'],
      dtype='object', length=980)

In [22]:
symbols_drop = list(symbols_no_close) + list(symbols_no_volume)  # combine symbols with no volume and no close
print(f'symbols_drop with duplicate symbols: {len(symbols_drop)}')
symbols_drop = list(set(symbols_drop))  # drop duplicate symbols
symbols_drop .sort()
print(f'symbols_drop with unique symbols: {len(symbols_drop)}')

symbols_drop with duplicate symbols: 1503
symbols_drop with unique symbols: 1253


In [23]:
df_a = df.drop(symbols_drop, axis=1, level=0)  # drop symbols from OHLCA df
df_a

A                                                  \
                  Open        High         Low       Close   Adj Close   
Date                                                                     
2021-11-16  157.050003  163.000000  156.960007  161.389999  160.362991   
2021-11-17  161.919998  161.919998  159.899994  161.539993  160.512024   
2021-11-18  162.529999  162.820007  160.399994  162.160004  161.128098   
2021-11-19  163.009995  165.070007  162.669998  164.300003  163.254471   
2021-11-22  165.000000  165.679993  162.779999  162.779999  161.744156   
...                ...         ...         ...         ...         ...   
2022-11-07  136.850006  138.080002  135.559998  138.029999  138.029999   
2022-11-08  138.500000  140.690002  136.600006  138.750000  138.750000   
2022-11-09  138.309998  139.419998  136.660004  137.000000  137.000000   
2022-11-10  142.869995  146.720001  142.529999  146.300003  146.300003   
2022-11-11  147.119995  149.740005  146.350006  148.309998  148.309998   

                              AA                                   ...  \
               Volume       Open       High        Low      Close  ...   
Date                                                               ...   
2021-11-16  1276300.0  49.860001  50.000000  47.959999  48.200001  ...   
2021-11-17   872400.0  48.910000  50.000000  47.619999  47.919998  ...   
2021-11-18  1475800.0  47.549999  47.830002  46.490002  47.700001  ...   
2021-11-19  1647600.0  47.310001  48.080002  46.689999  47.599998  ...   
2021-11-22  2110400.0  47.799999  49.700001  47.299999  48.509998  ...   
...               ...        ...        ...        ...        ...  ...   
2022-11-07  1213600.0  42.470001  42.790001  41.000000  41.509998  ...   
2022-11-08  1028100.0  42.180000  43.384998  41.459999  43.230000  ...   
2022-11-09   971200.0  42.529999  42.744999  40.340000  40.680000  ...   
2022-11-10  1591900.0  42.810001  45.490002  42.810001  43.830002  ...   
2022-11-11  1227500.0  45.259998  50.759998  45.040001  47.660000  ...   

                 ZUMZ                                        ZWS             \
                  Low      Close  Adj Close    Volume       Open       High   
Date                                                                          
2021-11-16  50.770000  52.509998  52.509998  190600.0  37.200001  38.189999   
2021-11-17  50.930000  51.000000  51.000000  232400.0  37.689999  37.900002   
2021-11-18  51.169998  53.680000  53.680000  240600.0  38.009998  38.029999   
2021-11-19  51.959999  52.040001  52.040001  277600.0  37.900002  38.320000   
2021-11-22  52.790001  54.630001  54.630001  427400.0  38.009998  38.169998   
...               ...        ...        ...       ...        ...        ...   
2022-11-07  22.020000  22.670000  22.670000  172900.0  23.049999  23.260000   
2022-11-08  22.219999  22.469999  22.469999  311800.0  23.170000  23.209999   
2022-11-09  20.639999  20.780001  20.780001  239800.0  22.260000  22.799999   
2022-11-10  21.780001  23.510000  23.510000  327600.0  22.920000  23.910000   
2022-11-11  23.670000  24.280001  24.280001  205000.0  23.590000  24.480000   

                                                        
                  Low      Close  Adj Close     Volume  
Date                                                    
2021-11-16  36.900002  37.779999  37.779999   788400.0  
2021-11-17  36.950001  37.709999  37.709999  1397700.0  
2021-11-18  37.360001  37.740002  37.740002   794900.0  
2021-11-19  37.689999  37.900002  37.900002   665300.0  
2021-11-22  37.110001  37.130001  37.130001  1235600.0  
...               ...        ...        ...        ...  
2022-11-07  22.809999  22.959999  22.959999  1587400.0  
2022-11-08  22.340000  22.600000  22.600000  2154600.0  
2022-11-09  21.910000  21.980000  21.980000  1013200.0  
2022-11-10  22.690001  23.440001  23.440001  2877700.0  
2022-11-11  23.549999  24.200001  24.200001  1534600.0  

[250 rows x 14376 columns]

In [24]:
df_c = df_close.iloc[look_back_days::]
df_c = df_c.drop(symbols_drop, axis=1)
df_c

,A,AA,AADI,AAL,AAN,AAON,AAP,AAPL,AAT,AAWW,...,ZIM,ZIP,ZLAB,ZM,ZNTL,ZS,ZTO,ZTS,ZUMZ,ZWS
Date,,,,,,,,,,,,,,,,,,,,,
2021-11-16,160.362991,47.845764,25.360001,19.840000,24.594210,76.634399,228.941742,150.128677,37.135880,87.989998,...,50.099998,32.150002,91.010002,264.489990,77.019997,353.750000,29.000000,219.826965,52.509998,37.779999
2021-11-17,160.512024,47.567814,25.480000,19.860001,24.447990,75.849075,225.891647,152.604309,36.827938,88.589996,...,52.799999,31.860001,90.680000,263.709991,79.279999,356.000000,28.820000,221.514252,51.000000,37.709999
2021-11-18,161.128098,47.349438,23.480000,19.389999,25.003626,76.137352,225.279678,156.959030,36.452637,89.720001,...,54.470001,31.350000,86.010002,255.750000,78.900002,365.649994,31.580000,225.178375,53.680000,37.740002
2021-11-19,163.254471,47.250172,24.770000,19.280001,24.623455,76.962456,223.414658,159.623596,36.346779,89.660004,...,57.169998,31.230000,84.150002,251.300003,77.629997,368.779999,32.610001,226.066940,52.040001,37.900002
2021-11-22,161.744156,48.153484,25.520000,19.629999,24.789171,77.469444,225.192276,160.090866,36.385273,92.559998,...,57.000000,29.850000,73.650002,242.279999,74.750000,343.859985,32.500000,223.980301,54.630001,37.130001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-11-07,138.029999,41.509998,13.030000,14.320000,9.840000,65.550003,183.089996,138.919998,27.990000,100.440002,...,24.629999,15.090000,32.029999,79.610001,23.549999,117.360001,19.330000,137.860001,22.670000,22.959999
2022-11-08,138.750000,43.230000,13.040000,14.250000,10.120000,74.750000,182.639999,139.500000,27.700001,100.099998,...,24.590000,14.800000,29.830000,78.309998,23.570000,119.930000,19.490000,136.210007,22.469999,22.600000
2022-11-09,137.000000,40.680000,13.410000,13.980000,10.020000,74.540001,175.350006,134.869995,27.219999,99.739998,...,23.170000,14.570000,28.570000,71.839996,18.389999,116.250000,18.680000,133.169998,20.780001,21.980000


In [25]:
print(f'symbols with no volume: {len(symbols_no_volume)}')
print(f'symbols with no close: {len(symbols_no_close)}')
print(f'unique symbols dropped from df_a and df_c: {len(symbols_drop)}')

symbols with no volume: 523
symbols with no close: 980
unique symbols dropped from df_a and df_c: 1253


In [86]:
# _periods = [-15, -30, -60, -120, -240]

perf_rank_dict = {}
_periods = [-15, -30, -60, -120, -240]
for _period in _periods:
  f_name = 'period' + str(_period)

  _df_c = df_c[_period::]
  symbols, period_yr, drawdown, UI, max_drawdown, returns_std, Std_UI, CAGR, CAGR_Std, CAGR_UI = \
      symb_perf_stats_vectorized(_df_c)
  caches_perf_stats_vect = []
  for symbol in symbols:
      date_first = drawdown.index[0].strftime('%Y-%m-%d')
      date_last = drawdown.index[-1].strftime('%Y-%m-%d')
      cache = (symbol, date_first, date_last, period_yr, CAGR[symbol],
              UI[symbol], Std_UI[symbol], CAGR_Std[symbol], CAGR_UI[symbol])
      # append performance data (tuple) to caches_perf_stats (list)
      caches_perf_stats_vect.append(cache)
  column_names = ['symbol', 'first date', 'last date', 'Year', 'CAGR',
                  'UI', 'Std/UI', 'CAGR/Std', 'CAGR/UI']

  # write symbols' performance stats to dataframe
  df_ps = pd.DataFrame(caches_perf_stats_vect, columns=column_names)
  df_ps['r_CAGR/UI'] = df_ps['CAGR/UI'].rank(ascending=False)
  df_ps['r_CAGR/Std'] = df_ps['CAGR/Std'].rank(ascending=False)
  df_ps['r_Std/UI'] = df_ps['Std/UI'].rank(ascending=False)
  
  _dict = {}
  cols_sort = ['r_CAGR/UI', 'r_CAGR/Std', 'r_Std/UI']
  print(f'{f_name} top 10 symbols')  
  for col in cols_sort:
    symbols_top_10 = df_ps.sort_values(by=[col]).head(10).symbol.values
    print(f'{col}: {symbols_top_10}')
    _dict[col] = symbols_top_10
    perf_rank_dict[f'{f_name}'] = _dict
  print(' ')

pickle_dump(perf_rank_dict, path_data_dump, 'perf_rank_dict')
print(f'Pickled perf_rank_dict to: {path_data_dump}perf_rank_dict\n')
print(f'perf_rank_dict:\n{perf_rank_dict}\n')


period-15 top 10 symbols
r_CAGR/UI: ['BAND' 'VCYT' 'SNCY' 'TXG' 'TSE' 'AFYA' 'ZLAB' 'NRIM' 'PIXY' 'NEOG']
r_CAGR/Std: ['BAND' 'VCYT' 'TXG' 'ZLAB' 'PIXY' 'TSE' 'SNCY' 'PRG' 'RUN' 'HASI']
r_Std/UI: ['NRIM' 'AFYA' 'KE' 'TWN' 'SNCY' 'AXON' 'GS' 'PZZA' 'ABMD' 'BVH']
 
period-30 top 10 symbols
r_CAGR/UI: ['KLXE' 'AKUS' 'IMVT' 'AVEO' 'MAC' 'PCVX' 'FORG' 'AXON' 'OLK' 'MAX']
r_CAGR/Std: ['KLXE' 'IMVT' 'AKUS' 'VCYT' 'MAC' 'PCVX' 'AXON' 'OLK' 'BAND' 'AVEO']
r_Std/UI: ['FORG' 'POSH' 'AKUS' 'AVEO' 'GILD' 'GRIN' 'AIMC' 'CRAI' 'MAX' 'SANM']
 
period-60 top 10 symbols
r_CAGR/UI: ['AKUS' 'AKRO' 'AERI' 'ADEA' 'TSBK' 'TDCX' 'ECOM' 'KLXE' 'AVEO' 'SGFY']
r_CAGR/Std: ['AKUS' 'AKRO' 'KLXE' 'TDCX' 'ADEA' 'RETA' 'IMVT' 'WFRD' 'SGML' 'TSBK']
r_Std/UI: ['AERI' 'ECOM' 'SGFY' 'AKRO' 'AKUS' 'FTCV' 'TSBK' 'ADEA' 'BLTS' 'PRTA']
 
period-120 top 10 symbols
r_CAGR/UI: ['RYTM' 'AVEO' 'AKRO' 'NUVL' 'ELF' 'PLRX' 'AKUS' 'TRDA' 'AERI' 'MYOV']
r_CAGR/Std: ['RYTM' 'AVEO' 'AKUS' 'NUVL' 'AKRO' 'AEHR' 'KLXE' 'IMVT' 'ELF' 'TRDA']